# Chapter 1: Scale from Zero to Millions of Users

## Single Server Setup

1. a user enters in a domain name, like facebook.com, into their browser that is read by a DNS (Domain name System)
2. the DNS finds the IP address (Internet Protocol) associated with that domain name and returns it to the browser or mobile app
3. using that IP address, an HTTP request is sent to the web server
4. that web server will return HTML or JSON for rendering

## Database

* if the user base grows, one server is not enough to handle all that data.
* we need multiple servers for web/mobile traffic to be handled (web tier) and we need to store that data in a database (data tier)
    - if we build it like this, we can scale both the web tier and data tier independently for our needs

### Which databases to use?

* Relational Database Management System (RDBMS) / SQL Databse
    - e.g. PostgreSQL, MySQL
    - stores data in tables and rows
    - able to perform join operations using SQL on those tables
    - you should use these when:
        * your data is inherently relational and structured
        * meaning that your data has a relationship with each other somehow
            - like a person will have an address or a phone number if you have their contact information
* Non-Relational databases aka NoSQL databases
    - e.g. MongoDB, DynamoDB
    -everything but SQL so key-value, graphs, columns, document stores
    - no join operations
    - you should use these when:
        * you need super-low latency
            - b/c you don't need join operations
            - much easier to scale horizontally
        * data is unstructured or there is no relational data
            - things like text files are unstructured because they're not predefined
            - a phone number has a predefined structure (area code - seven digit phone number) but a text file can contain pretty much anything. how do you qualify that?
    - you only need to serialize/deserialize data (JSON, XML, YAML)
    - need to store a massive amount of data
        * due to the unstructured nature of the data it collects

## Vertical Scaling vs. Horizontal Scaling

* Vertical scaling (scale up)
    - using one server but just adding more power to it (more CPU, RAM, etc)
    - great for low traffic and its pretty simple. just add more power when needed
    - however, that strength is also a disadvantage b/c there is a limit to how much power you can add to that one server
    - also, if it fails, your entire system fails
        * there's no redundancy or failsafe for it
* Horizontal scaling (scale out)
    - add more servers instead of more power
    - much more desirable for large scale applications

## Load Balancer

* a load balancer evenly distributes incoming traffic among web servers in a load-balanced set
* this helps to solve a couple of problems:
    1. if a web server is down, a load balancer can direct traffic to other working servers
    2. if there are way too many users trying to access your site, a load balancer can distribute them evenly

### The Setup:
* a user connects to a __public IP address__ of the load balancer that was retrieved by the DNS
    - the web servers themselves are no longer publically accessible
    - they can only be accessed using __private IP addresses__ that are known to themselves and the load balancer
    - this is for security reasons
* private IP addresses are IP addresses that can only be reachable between servers in the __same network__
* by using a load balancer, the web tier is now much more available
    - if a server goes down, the load balancer will direct traffic to a new web server and add a new web server to the set to replace the broken one
    - if the website traffic grows rapidly, we can add more web servers to handle the traffic

## Database Replication

* when we use a load balancer, we horizontally scale our web servers. however, we still only have one database that we do read/write operations on.
    - if that database happens to fail, then we're short out of luck
* however, we can employ Database Replication to support failover and redundancy so that it isn't a single point of failure

### Techniques:
* Master-Slave Replication
    - the Master is the original while the Slaves are the copies
    - the Master will only support __write operations (insert, delete, update)__, while the slaves only support __read operations__
    - most applications require a higher ratio of reads to writes so there are usually more Slaves to Masters
    - Advantages:
        * Better Performance:
            - since all writes go to Master and all reads go to Slaves, the reads can be done in parallel which is much faster
        * Reliability: if a database is destroyed, the data is still preserved since it was replicated
        * High Availability: even if a databse is offline/destroyed, other databases can take its place so that the website is still in operation
            - if there is only one slave database and it goes offline, the master database will take over read operations temporarily and a new slave database will be added to take the old one's place
            - if there are multiple slave databases, read operations will be directed to the healthy ones and a new slave databse will be added to replace the old
            - if the master database is offline, a slave database will be promoted to be the new master and all of the write operations will be directed to the new master. 
                * a new slave database is added to replace the promoted one
                * promoting a new master is pretty complicated since the slave database might not be up to date

### The Setup:
* user gets IP address of load balancer from DNS
* user connects to load balancer with the IP address
* HTTP request is routed to one of the web servers in the load-balancer set
* web server reads user data from a slave database
* web server routes any data-modifying operations to the master database (write, update, delete operations)

## Cache

* temporary storage that stores the result of expensive responses or frequently accessed data in memory so that they're quicker to retrieve
    - calling the database repeatedly can greatly affect performance. why not save that data

### Cache Tier
* the Cache Tier is a temporary data store layer that is much faster than a database
* Advantages:
    - better system performance
    - reduce database workloads
    - able to scale independently
    - pretty simple to interact
        * cache servers provide APIs for common programming languages

### Caching Strategies:
* Read-through Cache:
    - web server tries to read data from cache
    - if the cache doesn't have it, it'll query from the database and save it
    - it will return that data to the web server and any subsequent reads from the cache for that data will be much faster

### Considerations for using cache:
* Decide when to use cache: when data is read frequently but modified infrequently
    - cached data is stored in volatile (easily lost) memory and is not good for persisting data
    - if the cache fails or restarts, all of the data in that cache is lost
* Expiration Policy: once cached data is expired, it should be removed from the cache
    - no expiration = stored in memory permanently which is what we don't want
    - must find a good expiration time for the data
        * if expiration is too long, data becomes stale
        * if expiration is too short, must frequently read the database (performance issues)
* Consistency: keeping the database and the cache data in sync
    - inconsistency can happen when data is modified in the data-store but not in the cahce at the same time
    - can be pretty difficult to sync the two up
* Mitigating Failures: single cache server can be a potential single point of failure
    - to mitigate this, we can have multiple cache servers across different data centers
* Eviction Policy: when the cahce is full, any requests to add new items to the cache will also remove an item in the cache as well
    - this is called cache eviction
    - most popular: LRU (least recently used) cache
    - there are others as well like LFU (least frequently used) or FIFO (first in, first out)

## Content Delivery Network (CDN)

* network of geographically dispersed servers to deliver static content
    - they cache content like images, videos, CSS, JavaScript files, etc
* how it works on a high-level:
    - when a user visits a website, the CDN sever closest to them will deliver the static content

### Workflow:
1. User A tries to get image.png by using an image URL.
    - the domain for the URL is provided by the CDN provider
2. if the CDN server doesn't have the image.png in the cache, the CDN server will query for it from the original source, like a web server or some online storage like Amazon S3
3. the origin returns image.png to the CDN server which mihgt include an HTTP header Time-To-Live (TTL) which tells it how long the image should be cached for
4. CDN caches the image and returns it to User A and the image will remain cached until the TTL expires
5. User B sends a request to get the same image (image.png)
6. image is returned from the cache if the TTL is not expired

### Considerations of using a CDN:
* Cost: CDNs are run by 3rd party providers and you are charged for data transfers in/out of the CDN
    - caching infrequently used assets provides no significant benefits so they should be moved out of the CDN
* Setting an appropriate cache expiry
    - similar to the Expiration Poicy for Caches, you don't want your cached data expiry time to be too short or too long
    - too short = have to query original source for the data too frequently
    - too long = data in the cache is stale
* CDN fallback: how does your website/app cope with CDN failure?
    - if there is a CDN outage, the client should be able to request data from the original server
* Invalidating files: removing a file from a CDN before its TTL expires by doing the following:
    - invalidate CDN object using APIs provided by CDN vendors
    - using object versioning to serve different versions of the same object
        * requires you to add another parameter to the URL, e.g. image.png?v=2

## Stateless Web Tier

* way to scale the web tier horizontally
* moves state data (e.g. user session data) out of the web tier
    - can store session data in persistent storage like a relational database or NoSQL
    - each web server has access this state data

### Stateful Architecture

* stateful servers remember client data (state) from one request to the next
* for example: 
    - User A's session data/profile image stored in Server 1. 
    - to authenticate User A, the client has to be routed to Server 1 specifically for authentication to succeed
    - if Server 1 fails and instead the client for User A is routed to Server 2, then authentication would fail because their session data was not stored there
* the issue is that all requests from a specific client must be sent to a specific server.
    - if the route deviates, then authentication fails
    - this issue can be resolved with sticky sessions in load balancers but requires some more setup
    - also, it is more difficult to add/remove servers and challenging to handle server failures

### Stateless Architecture

* stateless servers don't remember client data (state) from one request to the next but instead fetches state data from a shared data store
* stateless systems are simpler, more robust (ability to cope with errors), and scalable
* how do we do this?
    - remove session data out of the web tier and store them in persistent data stores
    - e.g. relational database, memcached/Redis, NoSQL
    - in this case, NoSQL data store is chosen b/c it is easier to scale
* employing a stateless architecture, we can now scale the web tier by adding/removing web servers based on traffic load

## Data Centers

* data centers are used to improve availability and provide a better user experience across wider geographical areas
* based on a user's location, they are geoDNS-routed (geo-routed) to the closest data center
    - geoDNS is a DNS service that allows domain names to be resolved to IP addresses based on the location of a user
    - similar to how FFXIV data centers are
* if there is a data center outage, all traffic would be directed to a healthy data center

### The Setup:
* Traffic Redirection: need effective tools to redirect traffic to the correct data center
    - GeoDNS can be used to direct traffic to the nearest data center depending on where the user is located
* Data Synchronization: users from different regions could use different local databses/caches
    - in failover cases, traffic must be redirected from a failing data center to a working one but that data center might not have that data
    - so data might need to be replicated across multiple data centers for failover
        * e.g. Netflix does an asynchronous multi-data center replication to handle this
* Tests and Deployment: must test website/application at different locations
    - must use automated deployment tools to keep services consistent on all data centers

## Message Queue:

* message queue is a durable component stored in memory that supports asynchronous communication
    - it's a buffer that helps distribute asynchronous requests
    - it helps to decouple different components of your system so that they can be scaled independently
* basic architecture:
    - producers/publishers (input services) create the messages and publish them to the message queue
    - consumers/subscribers (other services/servers) connect to the queue and perform actions defined by those messages from the producers/publishers
* a producer can post a message to the queue that a consumer can process later if it's unavailable. and in turn, a consumer can process that message if the publisher is unavailable
    - they don't depend on each other to do their tasks. the message queue handles that
* for example:
    - your app supports photo customization (cropping, sharpening, blurring ,etc)
    - this customization takes time to complete
    - your web server can publish a photo processing job to the message queue
    - photo processing workers pick up those jobs from the message queue and asynchronously perform those tasks
* a message queue allows them to be independently scalable
    - if the queue becomes too big, more workers can be added to reduce processing time but if the queue is mostly empty, the number of workers can be reduced

## Logging, Metrics, and Automation

* having these is good practice for a smaller website but are a necessity for a large business with large systems
* Logging: monitoring error logs to identify errors and problems in the system
    - monitor them on a per server level
    - or aggregate them all into a centralized place to easily search/view them
* Metrics: collecting metrics can help gain business insight and understand health status of the system
    - Host level metrics: CPU, Memory, disk I/O, etc
    - Aggregated level metrics: performance of entire database tier, cache tier, etc
    - Key Business metrics: daily active users, retention, revenue, etc
* Automation: leveraging automation tools can improve productivity when a system gets big and complex
    - continuous integration is good practice so that code can be verified through automation to detect problems early on
    - can also automate the build, test, and deploy process so that developers can focus on the important tasks

## Database Scaling

* Vertical scaling: adding more power to an existing machine (CPU, RAM, DISK, etc.).
    - this is simpler but has its drawbacks:
        * you cannot continually add more power to your machine. there is a limit.
        * for a large user base, one server is just not enough
    - greater risk for single point of failures
    - overall cost is very high b/c powerful servers are more expensive
* Horizontal scaling:
    - also called sharding = separates large databases into smaller, more easily managed parts
        * has the same schema but the data on each shard is unique to it
    - for example:
        * user data is allocated to a database server based on user IDs
        * when you want to access data, you pass in the user ID into a hash function that identifies which shard it should look at
    - the most important factor when implementing a sharding strategy is the choice of the __sharding key or partition key__
        * a sharding key consists of 1+ columns that determine how data is distributed.
        * in our example above, we use user_id as the sharding key
        * __the sharding key should be chosen based on how well it can evenly distribute data__

### Challenges of Sharding (Horizontal Scaling):

* Resharding Data:
    1. when a single shard can no longer hold more data due to rapid growth
    2. some shards face shard exhaustion faster than others due to uneven data distribution
        - might need to change the sharding function and move data around again
        - consistent hashing can be used to solve this problem though
* Celebrity Problem (Hotspot Key Problem): excessive access to a specific shard can cause server overload
    - imagine if a bunch of celebrities' data ends up on the same shard. continual access to this shard would overload the server
    - to solve this, a shard might need to be created for each celebrity or might need to be partitioned further
* Join and De-normalization: when a database is sharded, it is hard to perform join operations across them
    - a solution to this is to de-normalize the database so that queries can be performed in a single table

## Millions of Users and Beyond:

* Keep web tier stateless
* Build redundancy at every tier
* Cache data as much as you can
* Support multiple data centers
* Host static assets in CDN
* Scale your data tier by sharding
* Split tiers into individual services
* Monitor your system and use automation tools

# Chapter 2: Back-Of-The-Envelope Estimation